In [1]:
import tensorflow as tf

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/',one_hot = True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784 

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width//(max_pool1_k*max_pool2_k)) * (input_height//(max_pool1_k*max_pool2_k)) * n_conv2

In [5]:
weights = {
    'wc1' : tf.Variable(tf.random_normal([conv1_k,conv1_k,input_channels,n_conv1])),
    'wc2' : tf.Variable(tf.random_normal([conv2_k,conv2_k,n_conv1,n_conv2])),
    'wh1' : tf.Variable(tf.random_normal([input_size_to_hidden,n_hidden])),
    'wo' : tf.Variable(tf.random_normal([n_hidden,n_out]))
}

biases = {
    'bc1' : tf.Variable(tf.random_normal([n_conv1])),
    'bc2' : tf.Variable(tf.random_normal([n_conv2])),
    'bh1' : tf.Variable(tf.random_normal([n_hidden])),
    'bo' : tf.Variable(tf.random_normal([n_out]))
}

In [6]:
def conv(x,weights,bias,strides = 1):
    out = tf.nn.conv2d(x,weights,padding = "SAME",strides = [1,strides,strides,1]) #applying the filter on images
    out = tf.nn.bias_add(out,bias) #adding the bias
    out = tf.nn.relu(out) #applying activation function
    return out


def maxpooling(x,k = 2):
    return tf.nn.max_pool(x,padding = "SAME",ksize = [1,k,k,1], strides = [1,k,k,1])#applying the filter


#strides = [1,k,k,1] or ksize = [1,k,k,1] means (no_of_images/no_of_examples, filter height, filter_width, no_of_channels)
#no_of_channels = 1, means that we want our filter to completely overlap the each image depth as discussed in the lecture/notes

In [10]:
#ksize is size of filter and strides is the stride length

In [11]:
#forward propogation function for cnn
#for better understanding draw the data flow of cnn 
def cnn(x,weights,biases,keep_prob):
    x = tf.reshape(x,shape = [-1,input_height,input_width,input_channels])
    conv1 = conv(x,weights['wc1'],biases['bc1'])
    conv1_pool = maxpooling(conv1,max_pool1_k)
    
    conv2 = conv(conv1_pool,weights['wc2'],biases['bc2'])
    conv2_pool = maxpooling(conv2,max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool,shape = [-1,input_size_to_hidden])
    hidden_input1 = tf.add(tf.matmul(hidden_input,weights['wh1']),biases['bh1']) #hidden input before applying activation function
    hidden_output_before_dropout_layer = tf.nn.relu(hidden_input1)
    hidden_output = tf.nn.dropout(hidden_output_before_dropout_layer, keep_prob)
    
    output = tf.add(tf.matmul(hidden_output,weights['wo']),biases['bo']) #using identity activation at the output
    return output

In [12]:
x = tf.placeholder('float',[None,input_pixels])
y = tf.placeholder(tf.int32,[None,n_out])
keep_prob = tf.placeholder("float")
pred = cnn(x,weights,biases,keep_prob)

In [14]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = pred,labels = y))

In [15]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
optimize = optimizer.minimize(cost)

In [16]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [17]:
batch_size = 100
for i in range(25):
    num_batches = int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x,batch_y = mnist.train.next_batch(batch_size)
        c,_ = sess.run([cost,optimize],feed_dict = {x:batch_x,y:batch_y,keep_prob:0.8})
        total_cost += c
    print(total_cost)
    

1368328.9211274385
30488.12668417759
19084.272375369444
15430.65945994109
12496.631876896497
12680.743846558324
13096.624466165067
12355.437388931918
12065.61652646428
9130.714053056668
8284.223000053913
7397.484305139887
7153.51960056297
7122.348416433062
6642.808814901653
4819.780151187144
3547.946520322701
3322.6520033195043
3886.6909524162174
3460.365853630199
3489.99842578459
2009.7434677220667
2793.778463858573
2851.3446922534135
2853.84682128991


In [18]:
pred_labels = tf.argmax(pred,1) #return the index with max_value
true_labels = tf.argmax(y,1)
correct_labels = tf.equal(pred_labels,true_labels)
pred_eval,correct_pred = sess.run([pred_labels,correct_labels],feed_dict = {x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
correct_pred.sum()
#getting 0.9833 accuracy

9833

In [ ]:
#regularisation using dropout layer
#Dropout layer is added after one of the hidden(or dense) layer
#Dropout layer has a keeping probability p,it means with probability (1-p), droput layer will ignore the unit
#if it is training,we will dropout
#if it is testing,we will not droput i.e., keeping probability,p = 1 